# Use Keras to recognize hand-written digits with Watson Machine Learning REST API

This notebook contains steps and code to demonstrate support of Deep Learning experiments in Watson Machine Learning Service. It introduces commands for getting data, training experiments, persisting pipelines, publishing models, deploying models and scoring.

Some familiarity with cURL is helpful. This notebook uses cURL examples.


## Learning goals

The learning goals of this notebook are:

-  Working with Watson Machine Learning experiments to train Deep Learning models.
-  Downloading computed models to local storage.
-  Online deployment and scoring of trained model.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)  
2.	[Experiment definition](#experiment_definition)  
3.  [Model definition](#model_definition)  
4.	[Experiment Run](#run)  
5.	[Historical runs](#runs)  
6.	[Deploy and Score](#deploy_and_score)  
7.	[Cleaning](#cleaning)   
8.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).
-  Create a <a href="https://console.bluemix.net/catalog/infrastructure/cloud-object-storage" target="_blank" rel="noopener no referrer">Cloud Object Storage (COS)</a> instance (a lite plan is offered and information about how to order storage can be found <a href="https://console.bluemix.net/docs/services/cloud-object-storage/basics/order-storage.html#order-storage" target="_blank" rel="noopener no referrer">here</a>). <br/>**Note: When using Watson Studio, you already have a COS instance associated with the project you are running the notebook in.**

You can find your COS credentials in COS instance dashboard under the **Service credentials** tab.
Go to the **Endpoint** tab in the COS instance's dashboard to get the endpoint information.

Authenticate the Watson Machine Learning service on IBM Cloud.

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.  

**NOTE:** You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.  


In [ ]:
%env API_KEY=...
%env WML_ENDPOINT_URL=...
%env WML_INSTANCE_CRN="fill out only if you want to create a new space"
%env WML_INSTANCE_NAME=...

%env COS_CRN="fill out only if you want to create a new space"
%env COS_ENDPOINT=...
%env COS_BUCKET=...
%env COS_ACCESS_KEY_ID=...
%env COS_SECRET_ACCESS_KEY=...
%env COS_API_KEY=...

%env SPACE_ID="fill out only if you have space already created"

%env DATAPLATFORM_URL=https://api.dataplatform.cloud.ibm.com   
%env AUTH_ENDPOINT=https://iam.cloud.ibm.com/oidc/token

<a id="wml_token"></a>
### Getting WML authorization token for further cURL calls

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" target="_blank" rel="noopener no referrer">Example of cURL call to get WML token</a>

In [2]:
%%bash --out token

curl -sk -X POST \
    --header "Content-Type: application/x-www-form-urlencoded" \
    --header "Accept: application/json" \
    --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
    --data-urlencode "apikey=$API_KEY" \
    "$AUTH_ENDPOINT" \
    | cut -d '"' -f 4

In [3]:
%env TOKEN=$token 

env: TOKEN=eyJraWQiOiIyMDIwMDgyMjE2NTYiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC01NTAwMDY3TkpEIiwiaWQiOiJJQk1pZC01NTAwMDY3TkpEIiwicmVhbG1pZCI6IklCTWlkIiwianRpIjoiMWQ5NzEyODItNTE3NC00MTI3LTk4YjctN2QwYzZiMTYwMGRiIiwiaWRlbnRpZmllciI6IjU1MDAwNjdOSkQiLCJnaXZlbl9uYW1lIjoiSmFuIiwiZmFtaWx5X25hbWUiOiJTb2x0eXNpayIsIm5hbWUiOiJKYW4gU29sdHlzaWsiLCJlbWFpbCI6Ikphbi5Tb2x0eXNpa0BpYm0uY29tIiwic3ViIjoiSmFuLlNvbHR5c2lrQGlibS5jb20iLCJhY2NvdW50Ijp7InZhbGlkIjp0cnVlLCJic3MiOiIxY2Y5ZjBlYjZhZjQ0NTlmOTQ3ODcxYmQzM2I0NzMxOCIsImZyb3plbiI6dHJ1ZX0sImlhdCI6MTU5ODQyNzIzNiwiZXhwIjoxNTk4NDMwODM2LCJpc3MiOiJodHRwczovL2lhbS50ZXN0LmNsb3VkLmlibS5jb20vb2lkYy90b2tlbiIsImdyYW50X3R5cGUiOiJ1cm46aWJtOnBhcmFtczpvYXV0aDpncmFudC10eXBlOmFwaWtleSIsInNjb3BlIjoiaWJtIG9wZW5pZCIsImNsaWVudF9pZCI6ImRlZmF1bHQiLCJhY3IiOjEsImFtciI6WyJwd2QiXX0.K8Bt1NNy0PoXn_KiWa6XRC3btmbY18GDCjyRntud3qXYSSCBdo_DA67RBBgsRlfppjGypwS3qivMggL5rjby9CBgdmEu1dqDpeqkSTXwJKpQ0WwzvPfzdlVMJs6NLGa8vvovgI0Gfyid7rCZqBx3OuZKyf-iI91AV2pd25AC5edwqID8aUU9AODqzGtqEkbRBcofJATlY0pf

<a id="space_creation"></a>
### Space creation
**Tip:** If you do not have `space` already created, please convert below three cells to `code` and run them.

First of all, you need to create a `space` that will be used in all of your further cURL calls. 
If you do not have `space` already created, below is the cURL call to create one.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_create" 
target="_blank" rel="noopener no referrer">Space creation</a>

Space creation is asynchronous. This means that you need to check space creation status after creation call.
Make sure that your newly created space is `active`.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_get" 
target="_blank" rel="noopener no referrer">Get space information</a>

<a id="experiment_definition"></a>
## 2. Experiment / optimizer configuration

<a id="training_connection"></a>
### Training data connection

Define connection information to COS bucket and training data npz file. This example uses the MNIST dataset. 

The dataset can be downloaded from [here](https://s3.amazonaws.com/img-datasets/mnist.npz).

**Action**: Upload training data to COS bucket and enter location information in the next cURL examples.

<a id="cos_token"></a>
### Get COS token
Retrieve COS token for further authentication calls.

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" 
target="_blank" rel="noopener no referrer">Retrieve COS authentication token</a>

In [4]:
%%bash --out cos_token

curl -s -X "POST" "$AUTH_ENDPOINT" \
     -H 'Accept: application/json' \
     -H 'Content-Type: application/x-www-form-urlencoded' \
     --data-urlencode "apikey=$COS_API_KEY" \
     --data-urlencode "response_type=cloud_iam" \
     --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
     | cut -d '"' -f 4

In [5]:
%env COS_TOKEN=$cos_token

env: COS_TOKEN=eyJraWQiOiIyMDIwMDgyMjE2NTYiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJpYW0tU2VydmljZUlkLWExNDFmNWNlLWVlZmYtNDU5Ny1iZTgyLTg3MTA1NTBhMjdjMyIsImlkIjoiaWFtLVNlcnZpY2VJZC1hMTQxZjVjZS1lZWZmLTQ1OTctYmU4Mi04NzEwNTUwYTI3YzMiLCJyZWFsbWlkIjoiaWFtIiwianRpIjoiMmQzOTJkZmMtYTAyOC00MWQ2LTkwM2MtZDczYjlkZTM5ZjQ3IiwiaWRlbnRpZmllciI6IlNlcnZpY2VJZC1hMTQxZjVjZS1lZWZmLTQ1OTctYmU4Mi04NzEwNTUwYTI3YzMiLCJuYW1lIjoiV0RQLVByb2plY3QtTWFuYWdlbWVudC1jODcxMGM1YS00YjM4LTRlMGYtYWQ3NC03MDM4Yzk4MTJjYzYiLCJzdWIiOiJTZXJ2aWNlSWQtYTE0MWY1Y2UtZWVmZi00NTk3LWJlODItODcxMDU1MGEyN2MzIiwic3ViX3R5cGUiOiJTZXJ2aWNlSWQiLCJhY2NvdW50Ijp7InZhbGlkIjp0cnVlLCJic3MiOiIxY2Y5ZjBlYjZhZjQ0NTlmOTQ3ODcxYmQzM2I0NzMxOCIsImZyb3plbiI6dHJ1ZX0sImlhdCI6MTU5ODQyNzI0NSwiZXhwIjoxNTk4NDMwODQ1LCJpc3MiOiJodHRwczovL2lhbS50ZXN0LmNsb3VkLmlibS5jb20vb2lkYy90b2tlbiIsImdyYW50X3R5cGUiOiJ1cm46aWJtOnBhcmFtczpvYXV0aDpncmFudC10eXBlOmFwaWtleSIsInNjb3BlIjoiaWJtIG9wZW5pZCIsImNsaWVudF9pZCI6ImRlZmF1bHQiLCJhY3IiOjEsImFtciI6WyJwd2QiXX0.dMwe9bpZikEZNZ6LNkQ9ZAmtKsj7vI_x41HW

<a id="cos_upload"></a>
### Upload file to COS
Upload your local dataset into your COS bucket

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-put-object" 
target="_blank" rel="noopener no referrer">Upload file to COS</a>

In [6]:
%%bash 

curl -sk -X PUT \
    --header "Authorization: Bearer $COS_TOKEN" \
    --header "Content-Type: application/octet-stream" \
    --data-binary "@../../../../data/mnist/mnist.npz" \
    "$COS_ENDPOINT/$COS_BUCKET/mnist.npz"

There should be an empty response when upload finished succesfully.

<a id="model_definition"></a>
## 3. Model definition 

This section provides samples about how to store model definition via cURL calls.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Model%20Definitions/model_definitions_create" 
target="_blank" rel="noopener no referrer">Store a model definition for Deep Learning experiment</a>

In [8]:
%%bash --out model_definition_payload

MODEL_DEFINITION_PAYLOAD='{"name": "mlp-model-definition", "space_id": "'"$SPACE_ID"'", "description": "mlp-model-definition", "tags": ["DL", "MNIST"], "version": "2.0", "platform": {"name": "python", "versions": ["3.6"]}, "command": "python3 mnist_mlp.py --trainImagesFile ${DATA_DIR}/train-images-idx3-ubyte.gz --trainLabelsFile ${DATA_DIR}/train-labels-idx1-ubyte.gz --testImagesFile ${DATA_DIR}/t10k-images-idx3-ubyte.gz --testLabelsFile ${DATA_DIR}/t10k-labels-idx1-ubyte.gz --learningRate 0.001 --trainingIters 6000"}'
echo $MODEL_DEFINITION_PAYLOAD | python -m json.tool

In [9]:
%env MODEL_DEFINITION_PAYLOAD=$model_definition_payload

env: MODEL_DEFINITION_PAYLOAD={
    "name": "mlp-model-definition",
    "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
    "description": "mlp-model-definition",
    "tags": [
        "DL",
        "MNIST"
    ],
    "version": "2.0",
    "platform": {
        "name": "python",
        "versions": [
            "3.6"
        ]
    },
    "command": "python3 mnist_mlp.py --trainImagesFile ${DATA_DIR}/train-images-idx3-ubyte.gz --trainLabelsFile ${DATA_DIR}/train-labels-idx1-ubyte.gz --testImagesFile ${DATA_DIR}/t10k-images-idx3-ubyte.gz --testLabelsFile ${DATA_DIR}/t10k-labels-idx1-ubyte.gz --learningRate 0.001 --trainingIters 6000"
}


In [10]:
%%bash --out model_definition_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_DEFINITION_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/model_definitions?version=2020-08-01"| grep '"id": ' | awk -F '"' '{ print $4 }'

In [11]:
%env MODEL_DEFINITION_ID=$model_definition_id

env: MODEL_DEFINITION_ID=fdfabc62-1244-49a2-ae34-905be56ddeaf


<a id="model_preparation"></a>
### Model preparation

Download files with keras code.  

<a href="https://github.com/pmservice/wml-sample-models/raw/master/keras/mnist/MNIST.zip" 
target="_blank" rel="noopener no referrer">Download MNIST.zip</a>

<a id="def_upload"></a>
### Upload model for the model definition

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Model%20Definitions/model_definitions_upload_model" 
target="_blank" rel="noopener no referrer">Upload model for the model definition</a>

In [12]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data-binary "@../../../../definitions/keras/mnist/MNIST.zip" \
    "$WML_ENDPOINT_URL/ml/v4/model_definitions/$MODEL_DEFINITION_ID/model?version=2020-08-01&space_id=$SPACE_ID" \
    | python -m json.tool

{
    "attachment_id": "22c5c6cf-9d78-47f2-a0b5-5985ec6134a8",
    "content_format": "native",
    "persisted": true
}


<a id="run"></a>
## 4. Experiment run

This section provides samples about how to trigger Deep Learning experiment via cURL calls.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_create" 
target="_blank" rel="noopener no referrer">Schedule a training job for Deep Learning experiment</a>

In [13]:
%%bash --out training_payload

TRAINING_PAYLOAD='{"training_data_references": [{"name": "training_input_data", "type": "s3", "connection": {"endpoint_url": "'"$COS_ENDPOINT"'", "access_key_id": "'"$COS_ACCESS_KEY_ID"'", "secret_access_key": "'"$COS_SECRET_ACCESS_KEY"'"}, "location": {"bucket": "'"$COS_BUCKET"'"}, "schema": {"id": "idmlp_schema", "fields": [{"name": "text", "type": "string"}]}}], "results_reference": {"name": "MNIST results", "connection": {"endpoint_url": "'"$COS_ENDPOINT"'", "access_key_id": "'"$COS_ACCESS_KEY_ID"'", "secret_access_key": "'"$COS_SECRET_ACCESS_KEY"'"}, "location": {"bucket": "'"$COS_BUCKET"'"}, "type": "s3"}, "tags": [{"value": "tags_mnist", "description": "dome MNIST"}], "name": "MNIST mlp", "description": "test training modeldef MNIST", "model_definition": {"id": "'"$MODEL_DEFINITION_ID"'", "command": "python3 mnist_mlp.py --trainImagesFile ${DATA_DIR}/train-images-idx3-ubyte.gz --trainLabelsFile ${DATA_DIR}/train-labels-idx1-ubyte.gz --testImagesFile ${DATA_DIR}/t10k-images-idx3-ubyte.gz --testLabelsFile ${DATA_DIR}/t10k-labels-idx1-ubyte.gz --learningRate 0.001 --trainingIters 6000", "hardware_spec": {"name": "K80", "nodes": 1}, "software_spec": {"name": "tensorflow_1.15-py3.6"}, "parameters": {"name": "MNIST mlp", "description": "Simple MNIST mlp model"}}, "space_id": "'"$SPACE_ID"'"}'
echo $TRAINING_PAYLOAD | python -m json.tool

In [14]:
%env TRAINING_PAYLOAD=$training_payload

env: TRAINING_PAYLOAD={
    "training_data_references": [
        {
            "name": "training_input_data",
            "type": "s3",
            "connection": {
                "endpoint_url": "https://s3.us-west.cloud-object-storage.test.appdomain.cloud",
                "access_key_id": "***",
                "secret_access_key": "***"
            },
            "location": {
                "bucket": "js-bucket"
            },
            "schema": {
                "id": "idmlp_schema",
                "fields": [
                    {
                        "name": "text",
                        "type": "string"
                    }
                ]
            }
        }
    ],
    "results_reference": {
        "name": "MNIST results",
        "connection": {
            "endpoint_url": "https://s3.us-west.cloud-object-storage.test.appdomain.cloud",
            "access_key_id": "***",
            "secret_access_key": "***"
        },
        "location": {
            "b

In [16]:
%%bash --out training_id

curl -sk -X  POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$TRAINING_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/trainings?version=2020-08-01" | awk -F'"id":' '{print $2}' | cut -c2-37

In [17]:
%env TRAINING_ID=$training_id

env: TRAINING_ID=f5cee2b3-e98f-46a8-bca6-c9c73b506520


<a id="training_details"></a>
### Get training details
Treining is an asynchronous endpoint. In case you want to monitor training status and details,
you need to use a GET method and specify which training you want to monitor by usage of training ID.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_get" 
target="_blank" rel="noopener no referrer">Get information about training job</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

### Get training status

In [25]:
%%bash

STATUS=$(curl -sk -X GET\
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01")
    
STATUS=${STATUS#*state\":\"}
STATUS=${STATUS%%\"*}
echo $STATUS

completed


Please make sure that training is completed before you go to the next sections.
Monitor `state` of your training by running above cell couple of times.

<a id="download_model"></a>
### Get selected model

Download a keras model object from the
Deep Learning training job.
As all of the trained pipeline models are stored in specified user COS bucket, you can download selected model
via COS UI or via following cURL call.
Please have in mind, that you need to specify COS bucket and model location in that bucket.
This information can be retrieved from the previous call output (GET training) under:
- entity -- status -- metrics  -- specific list entry -- intermediate_model -- location -- model

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-get-object" 
target="_blank" rel="noopener no referrer">Download file from COS</a>

In [26]:
%%bash --out model_path

PATH=$(curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01")


PATH=${PATH#*model\":\"}
MODEL_PATH="${PATH%%\"*}/mnist_cnn.h5"
echo $MODEL_PATH

In [27]:
%env MODEL_PATH=$model_path

env: MODEL_PATH=training-kpe1asNGR/model/mnist_cnn.h5


In [28]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $COS_TOKEN" \
    --output "mnist_cnn.h5" \
    "$COS_ENDPOINT/$COS_BUCKET/$MODEL_PATH"

If everything went right, you should be able to see your model under specified local path.

In [29]:
!ls -l mnist_cnn.h5

-rw-r--r--  1 jansoltysik  staff  5382376 Aug 26 09:43 mnist_cnn.h5


<a id="runs"></a>
## 5. Historical runs

In this section you will see cURL examples describing how to get historical training runs information.

Output should be similar to the output from training creation but you should see more trainings entries.  
Listing trainings:

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_list" 
target="_blank" rel="noopener no referrer">Get list of historical training jobs information</a>

In [ ]:
%%bash

HISTORICAL_TRAINING_LIMIT_TO_GET=2

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings?space_id=$SPACE_ID&version=2020-08-01&limit=$HISTORICAL_TRAINING_LIMIT_TO_GET" \
    | python -m json.tool

<a id="training_cancel"></a>
### Cancel training run

**Tip:** If you want to cancel your training, please convert below cell to `code`, specify training ID and run.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_delete" 
target="_blank" rel="noopener no referrer">Canceling training</a>

---

<a id="deploy_and_score"></a>
## 6. Deploy and Score

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

Before deployment creation, you need store your model in WML repository.
Please see below cURL call example how to do it. Remember that you need to
specify where your chosen model is stored in COS.

<a id="files_preparation"></a>
### Files preparation

Download all needed files (model.pickle, pipeline-model.json and schema.json) from COS.  
<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-get-object" 
target="_blank" rel="noopener no referrer">Download file from COS</a>

In [30]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $COS_TOKEN" \
    --output "mnist_cnn.h5" \
    "$COS_ENDPOINT/$COS_BUCKET/$MODEL_PATH"

Along with pipeline model, there is also a json file stored with experiment information.  
It will be needed during model repository store action.

Prepare compressed file with model and experiment information.

In [31]:
%%bash

tar -czvf mnist_cnn.h5.tar.gz mnist_cnn.h5

a mnist_cnn.h5


In [32]:
!ls -l mnist_cnn*

-rw-r--r--  1 jansoltysik  staff  5382376 Aug 26 09:43 mnist_cnn.h5
-rw-r--r--  1 jansoltysik  staff  4776509 Aug 26 09:43 mnist_cnn.h5.tar.gz


<a id="model_store"></a>
### Store Deep Learning model

Store information about your model to WML repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create" 
target="_blank" rel="noopener no referrer">Model storing</a>

In [33]:
%%bash --out model_payload

MODEL_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "name": "MNIST mlp", "description": "This is description", "type": "keras_2.2.5", "software_spec": {"name": "tensorflow_1.15-py3.6"}}'
echo $MODEL_PAYLOAD | python -m json.tool

In [34]:
%env MODEL_PAYLOAD=$model_payload

env: MODEL_PAYLOAD={
    "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
    "name": "MNIST mlp",
    "description": "This is description",
    "type": "keras_2.2.5",
    "software_spec": {
        "name": "tensorflow_1.15-py3.6"
    }
}


In [35]:
%%bash --out model_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [36]:
%env MODEL_ID=$model_id

env: MODEL_ID=5325c6a4-b728-4c56-91ed-2f82e4ae0168


<a id="model_content_upload"></a>
### Upload model content

Now you need to upload your model content into the WML repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_upload_content" 
target="_blank" rel="noopener no referrer">Upload model content</a>

In [37]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@mnist_cnn.h5.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | python -m json.tool

{
    "attachment_id": "2d3b1ad1-6a9d-46c0-aa6b-5f8a1705e6e5",
    "content_format": "native",
    "persisted": true
}


<a id="model_content_download"></a>
### Download model content

If you want to download your saved model, please make the following call.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_filtered_download" 
target="_blank" rel="noopener no referrer">Download model content</a>

In [38]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --output "mnist_cnn.h5.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/download?space_id=$SPACE_ID&version=2020-08-01"

In [39]:
!ls -l mnist_cnn.h5.tar.gz

-rw-r--r--  1 jansoltysik  staff  4776509 Aug 26 09:44 mnist_cnn.h5.tar.gz


<a id="deployment_creation"></a>
### Deployment creation

An Deep Learning online deployment creation is presented below.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_create" 
target="_blank" rel="noopener no referrer">Create deployment</a>

In [40]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "MNIST deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD | python -m json.tool

In [41]:
%env DEPLOYMENT_PAYLOAD=$deployment_payload

env: DEPLOYMENT_PAYLOAD={
    "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
    "name": "MNIST deployment",
    "description": "This is description",
    "online": {},
    "hardware_spec": {
        "name": "S"
    },
    "asset": {
        "id": "5325c6a4-b728-4c56-91ed-2f82e4ae0168"
    }
}


In [42]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p

In [43]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=388334e4-1e9f-4046-88cb-4fdc932928c6


<a id="deployment_details"></a>
### Get deployment details
As deployment API is asynchronous, please make sure your deployment is in `ready` state before going to the next points.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_get" 
target="_blank" rel="noopener no referrer">Get deployment details</a>

In [47]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "asset": {
            "id": "5325c6a4-b728-4c56-91ed-2f82e4ae0168"
        },
        "custom": {},
        "description": "This is description",
        "hardware_spec": {
            "id": "Not_Applicable",
            "name": "S",
            "num_nodes": 1
        },
        "name": "MNIST deployment",
        "online": {},
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
        "status": {
            "online_url": {
                "url": "https://wml-fvt.ml.test.cloud.ibm.com/ml/v4/deployments/388334e4-1e9f-4046-88cb-4fdc932928c6/predictions"
            },
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2020-08-26T07:44:38.900Z",
        "description": "This is description",
        "id": "388334e4-1e9f-4046-88cb-4fdc932928c6",
        "modified_at": "2020-08-26T07:44:38.900Z",
        "name": "MNIST deployment",
        "owner": "IBMid-5500067NJD",
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8e

<a id="input_score"></a>
### Prepare scoring input data
**Hint:** You may need to install numpy using following command `!pip install numpy`

In [21]:
import numpy as np

mnist_dataset = np.load('../../../../data/mnist/mnist.npz')
test_mnist = mnist_dataset['x_test']

In [22]:
image_1 = (test_mnist[0].ravel() / 255).tolist()
image_2 = (test_mnist[1].ravel() / 255).tolist()

<a id="webservice_score"></a>
### Scoring of a webservice
If you want to make a `score` call on your deployment, please follow a below method:

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_create" 
target="_blank" rel="noopener no referrer">Create deployment job</a>

In [48]:
%%bash -s "$image_1" "$image_2"

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data '{"space_id": "$SPACE_ID","input_data": [{"values": ['"$1"', '"$2"']}]}' \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2020-08-01" \
    | python -m json.tool

{
    "predictions": [
        {
            "fields": [
                "prediction",
                "prediction_classes",
                "probability"
            ],
            "values": [
                [
                    [
                        2.1821844678471258e-11,
                        9.103250375730454e-10,
                        3.332806031153268e-08,
                        6.23943890332157e-07,
                        3.506249855885456e-14,
                        6.874269209422579e-11,
                        2.1098052160165273e-17,
                        0.9999994039535522,
                        2.6728703694800515e-09,
                        1.1854215387074873e-08
                    ],
                    7,
                    [
                        2.1821844678471258e-11,
                        9.103250375730454e-10,
                        3.332806031153268e-08,
                        6.23943890332157e-07,
                        3.506249855885456

<a id="deployments_list"></a>
### Listing all deployments

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_list" 
target="_blank" rel="noopener no referrer">List deployments details</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="cleaning"></a>
## 7. Cleaning section

Below section is useful when you want to clean all of your previous work within this notebook.
Just convert below cells into the `code` and run them.

<a id="training_delete"></a>
### Delete training run
**Tip:** You can completely delete a training run with its metadata.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_delete" 
target="_blank" rel="noopener no referrer">Deleting training</a>

<a id="deployment_delete"></a>
### Deleting deployment
**Tip:** You can delete existing deployment by calling DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_delete" 
target="_blank" rel="noopener no referrer">Delete deployment</a>

<a id="model_delete"></a>
### Delete model from repository
**Tip:** If you want to completely remove your stored model and model metadata, just use a DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_delete" 
target="_blank" rel="noopener no referrer">Delete model from repository</a>

<a id="def_delete"></a>
### Delete model definition
**Tip:** If you want to completely remove your model definition, just use a DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Model%20Definitions/model_definitions_delete" 
target="_blank" rel="noopener no referrer">Delete model definition</a>

<a id="summary"></a>
## 8. Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use `cURL` calls to store, deploy and score a Deep Learning model in WML. 


### Authors

**Jan Sołtysik**, Intern in Watson Machine Learning at IBM

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.